In [1]:
import os
import xarray as xr
import pandas as pd
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
import hydromt
import numpy as np
from matplotlib.colors import LogNorm
sys.path.append(r'C:\Users\lelise\Documents\GitHub\flood_model_carolinas\syntheticTCs_cmpdfld')

In [2]:
os.chdir(r'Z:\Data-Expansion\users\lelise\projects\Carolinas_SFINCS\Chapter3_SyntheticTCs\04_RESULTS\ncep')
df = pd.read_csv(f'overland_flooded_area_table.csv', index_col=0)
df['AOI'] = [s.replace(" ","") for s in df['AOI']]
basins = df['AOI'].unique().tolist()
basins.remove('Domain')

In [3]:
# Load Vmax at landfall
landfall_vmax = pd.read_csv(r'Z:\Data-Expansion\users\lelise\projects\Carolinas_SFINCS\Chapter3_SyntheticTCs\02_DATA\NCEP_Reanalysis\tracks\ncep_landfall_vmax.csv',
                            index_col=0)
landfall_vmax.set_index('tc_id', inplace=True, drop=True)

# Load basin wind speed stats
basin_vmax = pd.read_csv(r'Z:\Data-Expansion\users\lelise\projects\Carolinas_SFINCS\Chapter3_SyntheticTCs\02_DATA\NCEP_Reanalysis\wind\basin_tc_windspd_stats.csv',
                         index_col=0)
print(basin_vmax.columns)

# Load basin rainfall stats
basin_rain = pd.read_csv(r'Z:\Data-Expansion\users\lelise\projects\Carolinas_SFINCS\Chapter3_SyntheticTCs\02_DATA\NCEP_Reanalysis\rain\basin_tc_precipitation_stats.csv',
                         index_col=0)
print(basin_rain.columns)

Index(['Pamlico_maxWS', 'Pamlico_meanWS', 'Pamlico_meanMaxWS',
       'Pamlico_meanWS_thresh', 'Pamlico_meanDirection', 'Neuse_maxWS',
       'Neuse_meanWS', 'Neuse_meanMaxWS', 'Neuse_meanWS_thresh',
       'Neuse_meanDirection', 'OnslowBay_maxWS', 'OnslowBay_meanWS',
       'OnslowBay_meanMaxWS', 'OnslowBay_meanWS_thresh',
       'OnslowBay_meanDirection', 'CapeFear_maxWS', 'CapeFear_meanWS',
       'CapeFear_meanMaxWS', 'CapeFear_meanWS_thresh',
       'CapeFear_meanDirection', 'LowerPeeDee_maxWS', 'LowerPeeDee_meanWS',
       'LowerPeeDee_meanMaxWS', 'LowerPeeDee_meanWS_thresh',
       'LowerPeeDee_meanDirection'],
      dtype='object')
Index(['Pamlico_totP', 'Pamlico_maxRR', 'Pamlico_meanRR',
       'Pamlico_meanRRthresh', 'Neuse_totP', 'Neuse_maxRR', 'Neuse_meanRR',
       'Neuse_meanRRthresh', 'OnslowBay_totP', 'OnslowBay_maxRR',
       'OnslowBay_meanRR', 'OnslowBay_meanRRthresh', 'CapeFear_totP',
       'CapeFear_maxRR', 'CapeFear_meanRR', 'CapeFear_meanRRthresh',
       'Lower

In [5]:
scenarios = ['Coastal','Runoff', 'Compound', 'Total_Flooded']
a = []
y = []

for i in range(len((basins))):
    # Organize the data into a single dataframe for plotting
    basin = basins[i]
    bdf = df[df['AOI'] == basin]
    stats = bdf.describe(percentiles=[0.9])
    combined = bdf.fillna(0)

    for k in range(len(scenarios)):
        scenario = scenarios[k]
        threshold = stats[scenario]['90%']
        combined2 = combined[combined[scenario] > threshold]
        combined2[scenario] = combined2[scenario].div(combined2['Total_Area'], axis=0)
        a.append(f'{basin} {scenario}')
        y.append(threshold)

stat_df = pd.DataFrame()
stat_df['scenario'] = a
stat_df['90PTC_thresh'] = y

print(stat_df)

                     scenario  90PTC_thresh
0             Pamlico Coastal      1556.168
1              Pamlico Runoff      3937.432
2            Pamlico Compound       122.208
3       Pamlico Total_Flooded      5550.304
4               Neuse Coastal       665.536
5                Neuse Runoff      4989.440
6              Neuse Compound        79.952
7         Neuse Total_Flooded      5612.520
8           OnslowBay Coastal       367.552
9            OnslowBay Runoff      1268.648
10         OnslowBay Compound        88.560
11    OnslowBay Total_Flooded      1646.616
12           CapeFear Coastal       127.944
13            CapeFear Runoff      5342.280
14          CapeFear Compound        74.900
15     CapeFear Total_Flooded      5531.008
16        LowerPeeDee Coastal       353.736
17         LowerPeeDee Runoff     13473.200
18       LowerPeeDee Compound       238.392
19  LowerPeeDee Total_Flooded     14026.376


In [ ]:
nrow = len(basins)
ncol = len(scenarios)

def plot_please(df, br, bws, var_r_label, var_ws_label, title, outfile):
    font = {'family': 'Arial', 'size': 10}
    mpl.rc('font', **font)
    mpl.rcParams.update({'axes.titlesize': 10})
    mpl.rcParams["figure.autolayout"] = True
    
    n_subplots = nrow * ncol
    first_in_row = np.arange(0, n_subplots, ncol)
    last_in_row = np.arange(ncol - 1, n_subplots, ncol)
    first_row = np.arange(0, ncol)
    last_row = np.arange(first_in_row[-1], n_subplots, 1)
    
    figsize = (7, 7.5)
    fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=figsize, tight_layout=True, sharex=True, sharey=True)
    for i in range(len((basins))):
        # Organize the data into a single dataframe for plotting
        basin = basins[i]
        bdf = df[df['AOI'] == basin]
        stats = bdf.describe(percentiles=[0.9])
        
        vmaxdf = bws[bws.index.isin(bdf.index)][basin]
        vmaxdf = vmaxdf.rename('wind')
        
        raindf = br[br.index.isin(bdf.index)][basin]
        raindf = raindf.rename('rain')
        
        combined = pd.concat(objs=[bdf, vmaxdf, raindf], axis=1, ignore_index=False)
        combined = combined.fillna(0)
    
    
        for k in range(len(scenarios)):
            scenario = scenarios[k]
            threshold = stats[scenario]['90%']
            combined2 = combined[combined[scenario] > threshold]
            combined2[scenario] = combined2[scenario].div(combined2['Total_Area'], axis=0)

            ax = axes[i][k]
            sc = ax.scatter(combined2['wind'], 
                            combined2['rain'], 
                            c=combined2[scenario], 
                            norm=LogNorm(),
                            cmap='Reds', marker='.', edgecolor='None'
                           )
            ax.grid(True)
            #ax.set_yscale('log')
            #ax.set_xscale('log')
            ax.set_axisbelow(True)
            
    axes = axes.flatten()
    
    for kk in range(nrow):
        axes[first_in_row[kk]].text(-0.35, 0.5, basins[kk],
                                    horizontalalignment='right',
                                    verticalalignment='center',
                                    rotation='vertical',
                                    transform=axes[first_in_row[kk]].transAxes)
        axes[first_in_row[kk]].set_ylabel(var_r_label)
        if kk == 2:
            pos0 = axes[first_in_row[kk]].get_position()  # get the original position
            cax1 = fig.add_axes([pos0.x1 + 0.65, pos0.y0, 0.05, pos0.height * 2.5])
            cbar1 = fig.colorbar(sc,
                                 cax=cax1,
                                 orientation='vertical',
                                 #ticks=[20, 40, 60, 80, 100],
                                 label='Area (sq.km)'
                                 )
            #cbar1.ax.set_yticklabels(labels=['25', '50', '75', '100', '125', '150'])
    
    for kk in range(ncol):
        axes[last_row[kk]].set_xlabel(var_ws_label)
        axes[first_row[kk]].set_title(scenarios[kk])
    
    #plt.colorbar(sc)
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.margins(x=0, y=0)
    fig.suptitle(title)
    plt.savefig(outfile, dpi=300, bbox_inches="tight")
    #plt.close()

In [ ]:
var_ws = 'meanWS'
select_columns = [x for x in basin_vmax if (var_ws in x) & ('thresh' not in x)]
print(select_columns)
bws = basin_vmax[select_columns]
bws.columns = [x.split('_')[0] for x in bws.columns]
print(bws.columns)

var_rain = 'meanRR'
select_columns = [x for x in basin_rain if (var_rain in x) & ('thresh' not in x)]
print(select_columns)
br = basin_rain[select_columns]
br.columns = [x.split('_')[0] for x in br.columns]
print(br.columns)
title = 'BasinAvg - Mean Rain Rate (mm.hr) vs. Mean Wind Speed (m.s)'

plot_please(df = df, 
            br = br, 
            bws = bws, 
            var_r_label='Rain Metric', 
            var_ws_label='Wind Speed Metric',
            title = title,
            outfile = f'{title}.png')

In [ ]:
var_ws = 'meanWS_thresh'
select_columns = [x for x in basin_vmax if var_ws in x]
print(select_columns)
bws = basin_vmax[select_columns]
bws.columns = [x.split('_')[0] for x in bws.columns]

var_rain = 'meanRRthresh'
select_columns = [x for x in basin_rain if (var_rain in x)]
print(select_columns)
br = basin_rain[select_columns]
br.columns = [x.split('_')[0] for x in br.columns]
title = 'BasinAvg - Mean Rain Rate above 5 mm.hr vs. Mean Wind Speed above 5 m.s - v3'
plot_please(df = df, 
            br = br, 
            bws = bws, 
            var_r_label='Rain Metric', 
            var_ws_label='Wind Speed Metric',
            title = title,
            outfile = f'{title}.png')

In [ ]:
var_ws = 'maxWS'
select_columns = [x for x in basin_vmax if var_ws in x]
print(select_columns)
bws = basin_vmax[select_columns]
bws.columns = [x.split('_')[0] for x in bws.columns]

var_rain = 'maxRR'
select_columns = [x for x in basin_rain if (var_rain in x)]
print(select_columns)
br = basin_rain[select_columns]
br.columns = [x.split('_')[0] for x in br.columns]
title = 'BasinWide - Max Rain Rate (mm.hr) vs. Max Wind Speed (m.s)'
plot_please(df = df, 
            br = br, 
            bws = bws, 
            var_r_label='Rain Metric', 
            var_ws_label='Wind Speed Metric',
            title = title,
            outfile = f'{title}.png')

In [ ]:
var_ws = 'meanWS'
select_columns = [x for x in basin_vmax if (var_ws in x) & ('thresh' not in x)]
print(select_columns)
bws = basin_vmax[select_columns]
bws.columns = [x.split('_')[0] for x in bws.columns]

var_rain = 'totP'
select_columns = [x for x in basin_rain if (var_rain in x)]
print(select_columns)
br = basin_rain[select_columns]
br.columns = [x.split('_')[0] for x in br.columns]
title = 'Total Precipitation (m) vs. BasinAvg Mean Wind Speed (m.s)'
plot_please(df = df, 
            br = br/1000, 
            bws = bws, 
            var_r_label='Rain Metric', 
            var_ws_label='Wind Speed Metric',
            title = title,
            outfile = f'{title}.png')

In [ ]:
var_ws = 'meanMaxWS'
select_columns = [x for x in basin_vmax if var_ws in x]
print(select_columns)
bws = basin_vmax[select_columns]
bws.columns = [x.split('_')[0] for x in bws.columns]

var_rain = 'totP'
select_columns = [x for x in basin_rain if (var_rain in x)]
print(select_columns)
br = basin_rain[select_columns]
br.columns = [x.split('_')[0] for x in br.columns]
title = 'Total Precipitation (m) vs. BasinAvg Mean Max Wind Speed (m.s)'
plot_please(df = df, 
            br = br/1000, 
            bws = bws, 
            var_r_label='Rain Metric', 
            var_ws_label='Wind Speed Metric',
            title = title,
            outfile = f'{title}.png')

In [ ]:
var_ws = 'meanDirection'
select_columns = [x for x in basin_vmax if var_ws in x]
basin_vmax2 = basin_vmax

# Modify values in the specified range (45 <= value <= 90) and make them negative
for col in select_columns:
    basin_vmax2[col] = basin_vmax2[col].apply(lambda x: -x if (x > 270) or (x < 90) else x)

columns_to_include = [col for col in basin_vmax2.columns if col not in select_columns]
for col in columns_to_include:
    b = col.split('_')[0]
    ref_col = f'{b}_meanDirection'
    basin_vmax2[col] = basin_vmax2.apply(lambda row: row[col] * (1 if row[ref_col] >= 0 else -1), axis=1)

In [ ]:
var_ws = 'meanWS'
select_columns = [x for x in basin_vmax2 if (var_ws in x) & ('thresh' not in x)]
print(select_columns)
bws = basin_vmax2[select_columns]
bws.columns = [x.split('_')[0] for x in bws.columns]

var_rain = 'meanRR'
select_columns = [x for x in basin_rain if (var_rain in x) & ('thresh' not in x)]
print(select_columns)
br = basin_rain[select_columns]
br.columns = [x.split('_')[0] for x in br.columns]
title = 'BasinAvg - Mean Rain Rate (mm.hr) vs. Mean Wind Speed (m.s) - v2'
plot_please(df = df, 
            br = br, 
            bws = bws, 
            var_r_label='Rain Metric', 
            var_ws_label='Wind Speed Metric',
            title = title,
            outfile = f'{title}.png')